In [1]:
import pandas as pd

df_applications_cleaned = pd.read_csv('cleaned_applications.csv')
df_loans_cleaned = pd.read_csv('cleaned_loans.csv')
df_defaults_cleaned = pd.read_csv('cleaned_defaults.csv')
df_branches_cleaned = pd.read_csv('cleaned_branches.csv')
df_customers_cleaned = pd.read_csv('cleaned_customers.csv')

In [2]:
merged_customer_loan_df = pd.merge(df_customers_cleaned, df_loans_cleaned, on='Customer_ID', how='inner')
display(merged_customer_loan_df.head())
display(merged_customer_loan_df.info())

Customer_ID      Full_Name  Contact_Number                  Email  \
0     C000003  Rhea Bhargava    910320534937  bahlelakshi@mall.info   
1     C000003  Rhea Bhargava    910320534937  bahlelakshi@mall.info   
2     C000004  Indrans Walia      6566372162     akadakia@yahoo.com   
3     C000004  Indrans Walia      6566372162     akadakia@yahoo.com   
4     C000004  Indrans Walia      6566372162     akadakia@yahoo.com   

                               Address  Age  Gender Marital_Status  \
0   67/386\nComar Nagar, Chapra 137669   62   Other         Single   
1   67/386\nComar Nagar, Chapra 137669   62   Other         Single   
2  88\nDhillon Ganj\nJamshedpur-816278   59  Female         Single   
3  88\nDhillon Ganj\nJamshedpur-816278   59  Female         Single   
4  88\nDhillon Ganj\nJamshedpur-816278   59  Female         Single   

  Employment_Status  Annual_Income  ...  Loan_Amount Interest_Rate  Loan_Term  \
0     Self-Employed        1118591  ...      2791233         14.66         60   
1     Self-Employed        1118591  ...      3402774          9.58         24   
2          Salaried         524075  ...      3822336         10.69         12   
3          Salaried         524075  ...      4989836         10.16         24   
4          Salaried         524075  ...      2347557          8.85         36   

  Disbursal_Date Repayment_Start_Date  Repayment_End_Date  Loan_Status  \
0     2024-06-30           2024-07-30          2029-06-30       Closed   
1     2021-01-10           2021-02-10          2023-01-10      Overdue   
2     2021-06-01           2021-07-01          2022-06-01       Closed   
3     2021-08-31           2021-09-30          2023-08-31      Overdue   
4     2023-08-23           2023-09-23          2026-08-23       Active   

   Overdue_Amount EMI_Amount Collateral_Details  
0               0      28882     Not Applicable  
1           44073      58437     Not Applicable  
2               0      13389           Property  
3               0      37450     Not Applicable  
4               0      65252           Property  

[5 rows x 25 columns]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90000 entries, 0 to 89999
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Customer_ID              90000 non-null  object 
 1   Full_Name                90000 non-null  object 
 2   Contact_Number           90000 non-null  int64  
 3   Email                    90000 non-null  object 
 4   Address                  90000 non-null  object 
 5   Age                      90000 non-null  int64  
 6   Gender                   90000 non-null  object 
 7   Marital_Status           90000 non-null  object 
 8   Employment_Status        90000 non-null  object 
 9   Annual_Income            90000 non-null  int64  
 10  Credit_Score             90000 non-null  int64  
 11  Region                   90000 non-null  object 
 12  Account_Tenure           90000 non-null  int64  
 13  Relationship_Manager_ID  90000 non-null  object 
 14  Loan_ID               

None

In [3]:
income_quantiles = merged_customer_loan_df['Annual_Income'].quantile([0.33, 0.66])
credit_quantiles = merged_customer_loan_df['Credit_Score'].quantile([0.33, 0.66])

def categorize_income(income):
    if income <= income_quantiles[0.33]:
        return 'Low Income'
    elif income <= income_quantiles[0.66]:
        return 'Medium Income'
    else:
        return 'High Income'

def categorize_credit_score(score):
    if score <= credit_quantiles[0.33]:
        return 'Low Credit'
    elif score <= credit_quantiles[0.66]:
        return 'Medium Credit'
    else:
        return 'High Credit'

merged_customer_loan_df['Income_Segment'] = merged_customer_loan_df['Annual_Income'].apply(categorize_income)
merged_customer_loan_df['Credit_Segment'] = merged_customer_loan_df['Credit_Score'].apply(categorize_credit_score)

merged_customer_loan_df['Customer_Segment'] = merged_customer_loan_df['Income_Segment'] + ' - ' + merged_customer_loan_df['Credit_Segment'] + ' - ' + merged_customer_loan_df['Loan_Status']

segment_counts = merged_customer_loan_df['Customer_Segment'].value_counts().reset_index()
segment_counts.columns = ['Customer_Segment', 'Count']
display(segment_counts)

Customer_Segment  Count
0        High Income - High Credit - Closed   3470
1       High Income - High Credit - Overdue   3465
2     High Income - Medium Credit - Overdue   3439
3        High Income - High Credit - Active   3434
4      Medium Income - High Credit - Active   3401
5         High Income - Low Credit - Active   3379
6        High Income - Low Credit - Overdue   3373
7         Low Income - High Credit - Active   3369
8         Low Income - High Credit - Closed   3359
9         High Income - Low Credit - Closed   3358
10   Medium Income - Medium Credit - Closed   3350
11    Medium Income - High Credit - Overdue   3346
12     High Income - Medium Credit - Active   3342
13     High Income - Medium Credit - Closed   3336
14     Medium Income - High Credit - Closed   3334
15      Low Income - Medium Credit - Closed   3323
16         Low Income - Low Credit - Active   3306
17         Low Income - Low Credit - Closed   3297
18      Low Income - Medium Credit - Active   3289
19        Low Income - Low Credit - Overdue   3277
20   Medium Income - Medium Credit - Active   3275
21     Medium Income - Low Credit - Overdue   3266
22      Medium Income - Low Credit - Closed   3257
23      Medium Income - Low Credit - Active   3255
24       Low Income - High Credit - Overdue   3249
25     Low Income - Medium Credit - Overdue   3232
26  Medium Income - Medium Credit - Overdue   3219

In [4]:
high_risk_segments = [segment for segment in merged_customer_loan_df['Customer_Segment'].unique()
                      if ('Low Income' in segment or 'Low Credit' in segment) and ('Overdue' in segment or 'Closed' in segment)]

high_value_segments = [segment for segment in merged_customer_loan_df['Customer_Segment'].unique()
                       if 'High Income' in segment and 'High Credit' in segment and 'Active' in segment]

merged_customer_loan_df['Is_High_Risk'] = merged_customer_loan_df['Customer_Segment'].isin(high_risk_segments)
merged_customer_loan_df['Is_High_Value'] = merged_customer_loan_df['Customer_Segment'].isin(high_value_segments)

print("High-Risk Customer Counts:")
display(merged_customer_loan_df['Is_High_Risk'].value_counts())

print("\nHigh-Value Customer Counts:")
display(merged_customer_loan_df['Is_High_Value'].value_counts())

High-Risk Customer Counts:


Is_High_Risk
False    57009
True     32991
Name: count, dtype: int64


High-Value Customer Counts:


Is_High_Value
False    86566
True      3434
Name: count, dtype: int64

In [5]:
repayment_analysis = merged_customer_loan_df.groupby('Customer_Segment').agg(
    Avg_Overdue_Amount=('Overdue_Amount', 'mean'),
    Overdue_Loan_Percentage=('Loan_Status', lambda x: (x == 'Overdue').mean() * 100),
    Avg_Interest_Rate=('Interest_Rate', 'mean')
).reset_index()

display(repayment_analysis)

Customer_Segment  Avg_Overdue_Amount  \
0        High Income - High Credit - Active        12665.428363   
1        High Income - High Credit - Closed        13121.661383   
2       High Income - High Credit - Overdue        12872.131313   
3         High Income - Low Credit - Active        12654.637467   
4         High Income - Low Credit - Closed        12727.582490   
5        High Income - Low Credit - Overdue        13084.041210   
6      High Income - Medium Credit - Active        12648.606822   
7      High Income - Medium Credit - Closed        12939.717026   
8     High Income - Medium Credit - Overdue        13006.250073   
9         Low Income - High Credit - Active        12796.511131   
10        Low Income - High Credit - Closed        12896.278654   
11       Low Income - High Credit - Overdue        12250.907048   
12         Low Income - Low Credit - Active        13063.126437   
13         Low Income - Low Credit - Closed        12849.273885   
14        Low Income - Low Credit - Overdue        12792.413793   
15      Low Income - Medium Credit - Active        12437.354515   
16      Low Income - Medium Credit - Closed        12622.080650   
17     Low Income - Medium Credit - Overdue        12848.862624   
18     Medium Income - High Credit - Active        12575.274331   
19     Medium Income - High Credit - Closed        12718.150570   
20    Medium Income - High Credit - Overdue        12774.138374   
21      Medium Income - Low Credit - Active        12728.686022   
22      Medium Income - Low Credit - Closed        12366.024870   
23     Medium Income - Low Credit - Overdue        12536.327618   
24   Medium Income - Medium Credit - Active        12406.179847   
25   Medium Income - Medium Credit - Closed        12519.485373   
26  Medium Income - Medium Credit - Overdue        13197.468158   

    Overdue_Loan_Percentage  Avg_Interest_Rate  
0                       0.0          11.032647  
1                       0.0          11.055784  
2                     100.0          11.089527  
3                       0.0          10.964359  
4                       0.0          10.986435  
5                     100.0          10.970783  
6                       0.0          11.004402  
7                       0.0          10.967257  
8                     100.0          10.967630  
9                       0.0          11.037486  
10                      0.0          10.922694  
11                    100.0          10.922496  
12                      0.0          10.978917  
13                      0.0          11.024401  
14                    100.0          11.016588  
15                      0.0          11.050213  
16                      0.0          11.050229  
17                    100.0          11.060142  
18                      0.0          10.979850  
19                      0.0          10.999598  
20                    100.0          10.960801  
21                      0.0          10.978298  
22                      0.0          11.075956  
23                    100.0          10.970220  
24                      0.0          11.020708  
25                      0.0          11.038755  
26                    100.0          11.017816

In [6]:
print("Customer Segment Summary:")
print("\nOverall Segment Distribution:")
display(segment_counts)

print("\nHigh-Risk Customer Characteristics:")
high_risk_repayment = repayment_analysis[repayment_analysis['Customer_Segment'].str.contains('Low Income|Low Credit')]
display(high_risk_repayment.sort_values(by='Overdue_Loan_Percentage', ascending=False).head())
print("High-risk segments are generally characterized by lower income or lower credit scores.")
print("They show higher average overdue amounts and overdue loan percentages compared to other segments.")

print("\nHigh-Value Customer Characteristics:")
high_value_repayment = repayment_analysis[repayment_analysis['Customer_Segment'].str.contains('High Income - High Credit - Active')]
display(high_value_repayment)
print("High-value segments are characterized by high income, high credit scores, and active loans.")
print("They typically have lower overdue amounts and 0% overdue loan percentage.")

print("\nComparison of Repayment Behaviors:")
print("High-risk segments, particularly those with 'Overdue' or 'Closed' loan statuses, exhibit significantly higher overdue amounts and overdue loan percentages.")
print("High-value segments, with 'Active' loan status, show excellent repayment behavior with no overdue loans.")
print("The average interest rates across segments do not show a clear pattern differentiating high-risk and high-value groups based on this analysis alone.")

Customer Segment Summary:

Overall Segment Distribution:


Customer_Segment  Count
0        High Income - High Credit - Closed   3470
1       High Income - High Credit - Overdue   3465
2     High Income - Medium Credit - Overdue   3439
3        High Income - High Credit - Active   3434
4      Medium Income - High Credit - Active   3401
5         High Income - Low Credit - Active   3379
6        High Income - Low Credit - Overdue   3373
7         Low Income - High Credit - Active   3369
8         Low Income - High Credit - Closed   3359
9         High Income - Low Credit - Closed   3358
10   Medium Income - Medium Credit - Closed   3350
11    Medium Income - High Credit - Overdue   3346
12     High Income - Medium Credit - Active   3342
13     High Income - Medium Credit - Closed   3336
14     Medium Income - High Credit - Closed   3334
15      Low Income - Medium Credit - Closed   3323
16         Low Income - Low Credit - Active   3306
17         Low Income - Low Credit - Closed   3297
18      Low Income - Medium Credit - Active   3289
19        Low Income - Low Credit - Overdue   3277
20   Medium Income - Medium Credit - Active   3275
21     Medium Income - Low Credit - Overdue   3266
22      Medium Income - Low Credit - Closed   3257
23      Medium Income - Low Credit - Active   3255
24       Low Income - High Credit - Overdue   3249
25     Low Income - Medium Credit - Overdue   3232
26  Medium Income - Medium Credit - Overdue   3219


High-Risk Customer Characteristics:


Customer_Segment  Avg_Overdue_Amount  \
5     High Income - Low Credit - Overdue        13084.041210   
11    Low Income - High Credit - Overdue        12250.907048   
14     Low Income - Low Credit - Overdue        12792.413793   
17  Low Income - Medium Credit - Overdue        12848.862624   
23  Medium Income - Low Credit - Overdue        12536.327618   

    Overdue_Loan_Percentage  Avg_Interest_Rate  
5                     100.0          10.970783  
11                    100.0          10.922496  
14                    100.0          11.016588  
17                    100.0          11.060142  
23                    100.0          10.970220

High-risk segments are generally characterized by lower income or lower credit scores.
They show higher average overdue amounts and overdue loan percentages compared to other segments.

High-Value Customer Characteristics:


Customer_Segment  Avg_Overdue_Amount  \
0  High Income - High Credit - Active        12665.428363   

   Overdue_Loan_Percentage  Avg_Interest_Rate  
0                      0.0          11.032647

High-value segments are characterized by high income, high credit scores, and active loans.
They typically have lower overdue amounts and 0% overdue loan percentage.

Comparison of Repayment Behaviors:
High-risk segments, particularly those with 'Overdue' or 'Closed' loan statuses, exhibit significantly higher overdue amounts and overdue loan percentages.
High-value segments, with 'Active' loan status, show excellent repayment behavior with no overdue loans.
The average interest rates across segments do not show a clear pattern differentiating high-risk and high-value groups based on this analysis alone.
